# 데이터 전처리 (1)

In [4]:
# 라이브러리 선언
import pandas as pd
import numpy as np

# 데이터 불러오기
csData = pd.read_csv("../../dataset/customerdata.csv", encoding = "ms949")
csDataOrigin = pd.read_csv("../../dataset/customerdata.csv", encoding = "ms949")

# 데이터 정보확인
csData.info()
csData.head()
print(csData.to_markdown)
csData.dtypes

# 함수를 활용한 조작 (복잡한 로직에도 적용이 )
# np.where를 활용한 조작 (복잡한 로직에 적용하기 어려움)
# loc 함수를 활용한 조작 (높은 가독성, 낮은 속도)
# map 함수를 활용한 조작 (머신/딥러닝에서 많이 사용)

In [10]:
## 1. 리스트에서 데이터프레임 생성하기

productList = ["aProduct", "bProduct", "cProduct", "dProduct"]
priceList = [1000, 2000, 3000, 4000]
zipList = zip(productList, priceList)
resultDF = pd.DataFrame(zipList, columns = ["prdname", "prdprice"])

In [19]:
## 2. 특정 항의 특정 수치 이상 조회하기

# 다중조건은 괄호를 통해 묶어준다.
resultDF.loc[resultDF.prdprice>2000]
csData2 = csData.loc[(csData.EMI >= 3) & (csData.AVGPRICE > 3500)]

In [24]:
## 3. 문자열 연산을 통한 특정 항 추출

targetID = "a13566"
howToUpper1 = csData.loc [csData.CUSTID == targetID.upper()]
howToUpper2 = csData.loc [csData.CUSTID.str.upper() == targetID.upper()]

In [35]:
## 4. 데이터 프레임 내 데이터 문자형 변환

# 중괄호가 없는 경우는 기존에 있는 컬럼내용을 변경할 때, 중괄호가 있는 경우는 새로운 컬럼을 만들어낼 때 사용 가능.
csData.EMI = csData.EMI.astype(str)
csData["test"] = csData["EMI"].astype(int)

In [37]:
## 5. 여러가지 컬럼을 동시에 변경할 때

csDataTypeStr = csData.astype({"CUSTID" : str, "AVGPRICE" : str, "EMI" : str, 
                           "DEVICECOUNT" : str, "PRODUCTAGE" : str, "CUSTTYPE" : str,
                           "test" : str})
targetEMI = "3"
targetPRI = "3500"
csData3 = csData.loc[(csData.EMI.astype(str) >= str(targetEMI)) &
                    (csData.AVGPRICE.astype(str) > str(targetPRI))]

In [51]:
## 6. 특정 항 배제하기

# 배제할 조건 설정 및 데이터프레임화 (추출)
angryCustomer = pd.DataFrame(list(csData.CUSTID[0:3]), columns = ["CSID"])

# 리스트화 시켜준 뒤 해당 리스트에 not in 조건 부여
angryList = list(angryCustomer.CSID)
csData.loc[~csData.CUSTID.isin(angryList)]

In [57]:
## 7. 인덱스를 통해 원하는 항 조회하기

# 1, 3항에 대해서만 조회하기
# 인덱스란, df 가장 좌측단에 위치하는 숫자에 해당한다.
specifyedIndex = [1,3]
test = csData.loc[specifyedIndex]

In [70]:
## 8. 타겟컬럼 정의 및 조회
targetcolumns = ["CUSTID", "AVGPRICE"]

# 컬럼으로 접근하기 (직관적임)
csDatatarget = csData.loc[ : , targetcolumns]
csDatamanual = csData.loc[ : , "CUSTID" : "EMI"]

# 인덱스로 접근하기 (코드의 형상관리 용이)
csDataIndex = csData.loc[1:4, : ]
csDataIndex2 = csData.iloc[0:1, : ]

In [73]:
## 9. 추출한 데이터 인덱스번호 초기화하기
emi3Data = csData.loc [csData.EMI ==3]

# 기존 인덱스 삭제
t1 = emi3Data.reset_index(drop = True)
# t1 = emi3Data.reset_index(drop = True, inplace = False)
# inplace 활용 시 코드를 더 가볍게 사용할 수 있다. (즉시 결과반영)

# 기존 인덱스 보여줌 (좌측에 인덱스 컬럼 추가)
t2 = emi3Data.reset_index(drop = False)

In [83]:
## 10. value의 범위별로 분류한 컬럼 추가하기

# 1보다 작으면 1, 아니면 0 반영 및 새로운 칼럼을 작성하는 엑셀식 (1, 1, 0)
# 각 자리에 여러가지 조건을 줄 수 있다. 예시는 else = 0의 자리에 조건을 추가한 것
t3 = csData["PRODUCTAGE_NEW"] = np.where(csData.PRODUCTAGE < 1, 1, 
                                         (np.where(csData.PRODUCTAGE < 2, 2, 3)))

# 검산식
csData.loc[(csData.PRODUCTAGE >= 1) & (csData.PRODUCTAGE_NEW != csData.PRODUCTAGE_NEW)]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,PRODUCTAGE_NEW


In [87]:
## 11. 함수식 선언을 통한 클러스터 적용

def valueCluster(inValue):
    outValue = inValue
    
    if inValue < 1:
        outValue = 1
    elif inValue < 2:
        outValue = 2
    else :
        outValue = 3
    return outValue

csData["PRODUCTAGE_NEW"] = csData.PRODUCTAGE.apply(valueCluster)

In [89]:
## 12. loc의 활용

# 선행조건이 없으면 작동하지 않고, 조건을 다중으로 부여해야 한다.
# 특정부분을 찾아내기 용이하다.
csData.loc[csData.PRODUCTAGE < 1, "PRODUCTAGE_NEW"] = 1
csData.loc[(csData.PRODUCTAGE >= 1) & (csData.PRODUCTAGE < 2), "PRODUCTAGE_NEW"] = 2
csData.loc[csData.PRODUCTAGE >= 2, "PRODUCTAGE_NEW"] = 3

In [91]:
## 13. map함수

featuresData = pd.read_csv("../../dataset/feature_regression_example.csv", encoding = "ms949")
ynMap = {"Y" : 1, "N" : 0}
featuresData["HOLIDAY_NEW"] = featuresData.HOLIDAY.map(ynMap)

In [ ]:
## 14. 컬럼간 문자값 연산

# CUSTID, EMI를 연산하려 함. 이 때 각 자료형은 str, int
# 자료형 불문, 대문자나 소문자 변환 시에는 중간에 str을 써준다.
# ex) csData.CUSTID.str.lower()
csData["NEW_COL"] = csData.CUSTID + "_" + csData.EMI.astype(str)